# MNIST with RELU

In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import random

tf.set_random_seed(333) # for reproducibility

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True) # one_hot 으로 y 데이터를 읽어옴


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [52]:
def logits(x,y,z):
    return tf.matmul(x,y)+z

# Problem of GradientDescent &Sigmoid

## Adam optimizer  
- GradientDescent 의 일종이지만 gradient 를 매개변수의 이동평균을 사용하여 계산
- step size 가 더 커지면서 적은 step size 로도 가능!
- 계산량 증가 및 모델 크기가 3배가 됨(평균, 분산 추가) 하지만 적은 step size로도 좋은 정확도를 만들어 낼 수 있음
- 즉 gradient descnet의 에러보다 훨씬 빠르게 줄어듦
- ex) Adam step : 400~800 but, Gradient Descent : 2000

## RELU
- 앞으로 갈수록 w 의 값이 결과에 거의 영향을 안미치는 문제 해결
- 0에서 1에 국한되지 않고 0~inf로

In [60]:
# tensorboard reset
tf.reset_default_graph()

#paramiters
learning_rate=0.01
batch_size = 100
training_epochs = 50
num = int(mnist.train.num_examples/batch_size)

# input place holders
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

with tf.name_scope("layer1"):
    w1 = tf.Variable(tf.random_normal([784,256]))
    b1 = tf.Variable(tf.random_normal([256]))
    layer1 = tf.nn.relu(logits(x,w1,b1))

with tf.name_scope("layer2"):
    w2 = tf.Variable(tf.random_normal([256,256]))
    b2 = tf.Variable(tf.random_normal([256]))
    layer2 = tf.nn.relu(logits(layer1, w2, b2))

with tf.name_scope("hypothesis"):
    w3 = tf.Variable(tf.random_normal([256,10]))
    b3 = tf.Variable(tf.random_normal([10]))
    hypothesis = logits(layer2, w3,b3)

#define cost/Loss $ optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis,
                                    labels = y))
tf.summary.scalar("cost",cost)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

# tensorboard node
summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./MNIST_graph")

sess=tf.Session()
sess.run(tf.global_variables_initializer())

# add session graph
writer.add_graph(sess.graph)

for epoch in range(training_epochs):
    avg_cost=0
    for step in range(num):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        s,c,_ = sess.run([summary, cost, optimizer], 
                         feed_dict={x:batch_x, y:batch_y})
        writer.add_summary(s, global_step=step+epoch*mnist.train.num_examples)
        avg_cost+=c/num
    print("average_cost : {:.5f}".format(c))

# accuacy
accuracy = tf.reduce_mean(tf.cast(tf.equal(
    tf.argmax(hypothesis, 1), tf.argmax(y,1)), dtype=tf.float32))

# tf.argmax(hypothesis, 1) 은 int 형식 but, y는 float 형식이기 때문에
# y 에도 tf.argmax(y,1) 을 통해 형식을 맞춰줌!!!

a= sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
print("accuacy : {:.5%}".format(a))
# => training 시켜놓은 모델을 통해 예측하기만 하면 되기 때문에
# optimizer node를 실행시킬 이유가 전혀 없음!!!!!

# anaconda 에서 tensorboard 실행 : tensorboard --logdir=./directory

average_cost : 14.18374
average_cost : 10.14060
average_cost : 9.86606
average_cost : 0.44760
average_cost : 5.00859
average_cost : 4.41425
average_cost : 0.82572
average_cost : 3.93973
average_cost : 0.69115
average_cost : 0.22258
average_cost : 2.09496
average_cost : 2.06805
average_cost : 2.91023
average_cost : 4.35140
average_cost : 2.56475
average_cost : 1.58754
average_cost : 0.05158
average_cost : 0.09049
average_cost : 2.03332
average_cost : 0.33566
average_cost : 0.87890
average_cost : 0.81062
average_cost : 0.55758
average_cost : 0.70882
average_cost : 0.00032
average_cost : 0.59343
average_cost : 0.19194
average_cost : 0.04823
average_cost : 0.04123
average_cost : 0.45648
average_cost : 0.03428
average_cost : 0.11447
average_cost : 0.47420
average_cost : 0.01584
average_cost : 0.02121
average_cost : 0.00910
average_cost : 0.20934
average_cost : 0.04502
average_cost : 0.61712
average_cost : 0.07573
average_cost : 0.07121
average_cost : 0.05173
average_cost : 0.00720
average_c

# Problem of Initializaition

# xavier initialization tensorflow

- 몇개의 입력값과 출력값을 갖는지에 따라 초깃값을 결정

In [63]:
# tensorboard reset
tf.reset_default_graph()

#paramiters
learning_rate=0.01
batch_size = 100
training_epochs = 50
num = int(mnist.train.num_examples/batch_size)

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

with tf.name_scope("layer1"):
    w1 = tf.get_variable("w1", shape = [784,256],
                        initializer = tf.contrib.layers.xavier_initializer())
    b1 = tf.get_variable("b1", shape = [256],
                        initializer = tf.contrib.layers.xavier_initializer())
    l1 = tf.nn.relu(logits(x,w1,b1))
    
with tf.name_scope("layer2"):
    w2 = tf.get_variable("w2", shape = [256,256],
                        initializer = tf.contrib.layers.xavier_initializer())
    b2 = tf.get_variable("b2", shape = [256],
                        initializer = tf.contrib.layers.xavier_initializer())
    l2 = tf.nn.relu(logits(l1,w2,b2))
    
with tf.name_scope("layer3"):
    w3 = tf.get_variable("w3", shape = [256,10],
                        initializer = tf.contrib.layers.xavier_initializer())
    b3 = tf.get_variable("b3", shape = [10],
                        initializer = tf.contrib.layers.xavier_initializer())
    hypothesis = logits(l2,w3,b3)
    
#define cost/Loss $ optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis,
                                    labels = y))
tf.summary.scalar("cost",cost)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

# tensorboard node
summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./MNIST_graph")

sess=tf.Session()
sess.run(tf.global_variables_initializer())

# add session graph
writer.add_graph(sess.graph)

for epoch in range(training_epochs):
    avg_cost=0
    for step in range(num):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        s,c,_ = sess.run([summary, cost, optimizer], 
                         feed_dict={x:batch_x, y:batch_y})
        writer.add_summary(s, global_step=step+epoch*mnist.train.num_examples)
        avg_cost+=c/num
    print("average_cost : {:.5f}".format(c))

# accuacy
accuracy = tf.reduce_mean(tf.cast(tf.equal(
    tf.argmax(hypothesis, 1), tf.argmax(y,1)), dtype=tf.float32))

# tf.argmax(hypothesis, 1) 은 int 형식 but, y는 float 형식이기 때문에
# y 에도 tf.argmax(y,1) 을 통해 형식을 맞춰줌!!!

a= sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
print("accuacy : {:.5%}".format(a))
# => training 시켜놓은 모델을 통해 예측하기만 하면 되기 때문에
# optimizer node를 실행시킬 이유가 전혀 없음!!!!!

# anaconda 에서 tensorboard 실행 : tensorboard --logdir=./directory

average_cost : 0.18663
average_cost : 0.09842
average_cost : 0.06180
average_cost : 0.13780
average_cost : 0.04258
average_cost : 0.23405
average_cost : 0.00801
average_cost : 0.03018
average_cost : 0.00273
average_cost : 0.06974
average_cost : 0.07376
average_cost : 0.04162
average_cost : 0.04981
average_cost : 0.02201
average_cost : 0.07055
average_cost : 0.06270
average_cost : 0.04189
average_cost : 0.02749
average_cost : 0.13337
average_cost : 0.02748
average_cost : 0.02261
average_cost : 0.05832
average_cost : 0.01765
average_cost : 0.07701
average_cost : 0.03051
average_cost : 0.05037
average_cost : 0.00451
average_cost : 0.00376
average_cost : 0.00053
average_cost : 0.00875
average_cost : 0.05680
average_cost : 0.04006
average_cost : 0.01512
average_cost : 0.01581
average_cost : 0.06060
average_cost : 0.03154
average_cost : 0.04790
average_cost : 0.01118
average_cost : 0.00239
average_cost : 0.02470
average_cost : 0.00972
average_cost : 0.01610
average_cost : 0.01936
average_cos

 # Problem of Overfitting

# Dropout
- 랜덤으로 몇개의 layer을 버린 뒤 실행
- 실행할 때마다 랜덤값은 달라짐
- 몇개의 입력값을 버리는 것과 같기 때문에 test set 에서는 keep_prod 를 1로 설정해주어야 함!

In [68]:
# tensorboard reset
tf.reset_default_graph()

#paramiters
learning_rate=0.01
batch_size = 100
training_epochs = 50
num = int(mnist.train.num_examples/batch_size)

# dropout (keep_prob) rate 0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

with tf.name_scope("layer1"):
    w1 = tf.get_variable("w1", shape = [784,512],
                        initializer = tf.contrib.layers.xavier_initializer())
    b1 = tf.get_variable("b1", shape = [512],
                        initializer = tf.contrib.layers.xavier_initializer())
    l1 = tf.nn.relu(logits(x,w1,b1))
    l1 = tf.nn.dropout(l1, keep_prob = keep_prob)
    
with tf.name_scope("layer2"):
    w2 = tf.get_variable("w2", shape = [512,512],
                        initializer = tf.contrib.layers.xavier_initializer())
    b2 = tf.get_variable("b2", shape = [512],
                        initializer = tf.contrib.layers.xavier_initializer())
    l2 = tf.nn.relu(logits(l1,w2,b2))
    l2 = tf.nn.dropout(l2, keep_prob = keep_prob)
    
with tf.name_scope("layer3"):
    w3 = tf.get_variable("w3", shape = [512,10],
                        initializer = tf.contrib.layers.xavier_initializer())
    b3 = tf.get_variable("b3", shape = [10],
                        initializer = tf.contrib.layers.xavier_initializer())
    hypothesis = logits(l2,w3,b3)
    
#define cost/Loss $ optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis,
                                    labels = y))
tf.summary.scalar("cost",cost)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

# tensorboard node
summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./MNIST_graph")

sess=tf.Session()
sess.run(tf.global_variables_initializer())

# add session graph
writer.add_graph(sess.graph)

for epoch in range(training_epochs):
    avg_cost=0
    for step in range(num):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        s,c,_ = sess.run([summary, cost, optimizer], 
                         feed_dict={x:batch_x, y:batch_y, keep_prob : 0.7})
        writer.add_summary(s, global_step=step+epoch*mnist.train.num_examples)
        avg_cost+=c/num
    print("average_cost : {:.5f}".format(c))

# accuacy
accuracy = tf.reduce_mean(tf.cast(tf.equal(
    tf.argmax(hypothesis, 1), tf.argmax(y,1)), dtype=tf.float32))

# tf.argmax(hypothesis, 1) 은 int 형식 but, y는 float 형식이기 때문에
# y 에도 tf.argmax(y,1) 을 통해 형식을 맞춰줌!!!

a= sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels,
                                 keep_prob:1})
print("accuacy : {:.5%}".format(a))
# => training 시켜놓은 모델을 통해 예측하기만 하면 되기 때문에
# optimizer node를 실행시킬 이유가 전혀 없음!!!!!

# anaconda 에서 tensorboard 실행 : tensorboard --logdir=./directory

average_cost : 0.26704
average_cost : 0.17772
average_cost : 0.37853
average_cost : 0.20183
average_cost : 0.19908
average_cost : 0.11749
average_cost : 0.12567
average_cost : 0.28998
average_cost : 0.31826
average_cost : 0.20977
average_cost : 0.22589
average_cost : 0.22677
average_cost : 0.21671
average_cost : 0.14707
average_cost : 0.13988
average_cost : 0.19199
average_cost : 0.16821
average_cost : 0.18863
average_cost : 0.32551
average_cost : 0.23350
average_cost : 0.24057
average_cost : 0.50760
average_cost : 0.40387
average_cost : 0.38214
average_cost : 0.28324
average_cost : 0.06814
average_cost : 0.17852
average_cost : 0.25214
average_cost : 0.16326
average_cost : 0.23111
average_cost : 0.28513
average_cost : 0.17517
average_cost : 0.14888
average_cost : 0.12256
average_cost : 0.29147
average_cost : 0.09814
average_cost : 0.17129
average_cost : 0.18240
average_cost : 0.04158
average_cost : 0.17980
average_cost : 0.19433
average_cost : 0.09985
average_cost : 0.04079
average_cos

# Optimizers
- tf.train.AdadeltaOptimizer
- tf.train.AdagradOptimizer
- tf.train.AdagradDAOptimizer
- tf.train.MomentumOptimizer
- tf.train.AdamOptimizer           => 적은 step 으로도 cost 를 많이 줄일 수 있음
- tf.train.FtrlOptimizer
- tf.train.ProximalGradientDescentOptimizer
- tf.train.ProximalAdagradOptimizer
- tf.train.RMSPropOptimizer

# Batch Normalization
- 학습 속도 개선(큰 learning rate으로 적은 step size 가능)
- 가중치 초깃값의 의존도 줄어듦(평균0, 분산1의 output)
- overfitting 의 문제해결(dropout / regulization 대체 가능)
- Gradient Vanishing 문제 해결

<img src = https://camo.githubusercontent.com/71f966091fd4696bc534ec9963806b65e962a7de/68747470733a2f2f6b7261747a6572742e6769746875622e696f2f696d616765732f626e5f6261636b706173732f626e5f616c676f726974686d2e504e47 width = 300><br>

- train set => 각각의 input 이 다르기 때문에 그 때마다 batch normalization
- test set  => 빠르게 속도를 판단해야 하기 때문에 고정된 평균과 표준편차를 사용


### 1. Load Libarary

In [70]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [73]:
print(mnist.train.images.shape)
print(mnist.train.labels.shape)

(55000, 784)
(55000, 10)


### Define Model & Solver Class

- 객체지향모델로 모델 구축
- model과 solver 을 구축

In [ ]:
class model:
    def __init__(self, name, input_dim, output_dim, hidden_dims = [32,32],
                use_batchnorm=True, activation_fn = tf.nn.relu,
                oprimizer = tf.train.AdamOptimizer, lr=0.01):
        with tf.variable_scope(name): # tf.name_scope 는 그냥 Variable 에만 적용
           # tf.variable_scope 은 get_variable 과 Variable 모두에 적용
    # tf.variable_scope와 tf.get_variable 의 조합을 통해 텐서를 reuse 할 수도 있음
            net = tf.layers.dense